Import

In [ ]:
# Ensure repository root (with 'Source/ai') is on sys.path
from pathlib import Path
import sys

project_root = next((p for p in [Path.cwd(), *Path.cwd().parents] if (p / 'Source' / 'ai').exists()), None)
if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain.tools import Tool
from langgraph.graph import END, StateGraph
import operator
import requests
from Source.ai.Multi_Agent.Source.Main.Tools import poem_tools, sentiment_tools, travel_tools, weather_tools
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_1 import Coordinator_Agent_1, Flight_Agent_1, Hotel_Agent_1, Travel_Agent_1
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_2 import Coordinator_Agent_2, Flight_Agent_2, Hotel_Agent_2, Travel_Agent_2
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_3 import Coordinator_Agent_3, Flight_Agent_3, Hotel_Agent_3, Travel_Agent_3, Aggregator_Agent_3
from typing import TypedDict, Annotated, List, Any, Dict, Literal
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from Source.ai.Multi_Agent.Source.Main.Memory.memory.memory import memory_manager
from Source.ai.Multi_Agent.Source.Main.Memory.memory.long_term_memory import long_term_memory
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import uuid
import os

c:\Users\minhv\anaconda3\envs\pytorch-env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
e:\Project_NguyenMinhVu_2211110063\Source\ai\Multi_Agent\Source\Main\Tools\poem_tools.py:14: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3:8b") # <-- Sử dụng model bạn đã kéo về, ví dụ "llama3", "mistral"


Load model

In [2]:
load_dotenv()

llm = ChatOllama(model="llama3:8b") # <-- Sử dụng model bạn đã kéo về, ví dụ "llama3", "mistral"

class AgentState(TypedDict):
    input: str
    messages: Annotated[List[str], operator.add]
    
prompt = hub.pull("hwchase17/react")

Load tools

In [3]:
weatherllm_agent = create_react_agent(llm, [weather_tools.weatherllm_tool], prompt) 
weatherllm_agent_executor = AgentExecutor(agent=weatherllm_agent, tools=[weather_tools.weatherllm_tool], verbose=True, handle_parsing_errors=True) 

def call_weatherllm_agent(state: AgentState):
    result = weatherllm_agent_executor.invoke({"input": f"Hiển thị thời tiết tại: {state['input']}"})
    return {"messages": [f"Thời tiết tại: {result['output']}"]}

sentiment_agent = create_react_agent(llm, [sentiment_tools.sentiment_tool], prompt) 
sentiment_agent_executor = AgentExecutor(agent=sentiment_agent, tools=[sentiment_tools.sentiment_tool], verbose=True, handle_parsing_errors=True)

def call_sentiment_agent(state: AgentState):
    result = sentiment_agent_executor.invoke({"input": f"Phân tích cảm xúc của đoạn text sau: {state['input']}"})
    return {"messages": [f"Phân tích cảm xúc: {result['output']}"]} 

poet_agent = create_react_agent(llm, [poem_tools.poem_tool], prompt) 
poet_agent_executor = AgentExecutor(agent=poet_agent, tools=[poem_tools.poem_tool], verbose=True, handle_parsing_errors=True) 
def call_poet_agent(state: AgentState):
    result = poet_agent_executor.invoke({"input": f"Hãy viết một bài thơ về: {state['input']}"})
    return {"messages": [f"Bài thơ: {result['output']}"]}

# weatherapi_agent = create_react_agent(llm, [weather_tools.weatherapi_tool], prompt) 
# weatherapi_agent_executor = AgentExecutor(agent=weatherapi_agent, tools=[weather_tools.weatherapi_tool], verbose=True, handle_parsing_errors=True) 

# def call_weatherapi_agent(state: AgentState):
#     result = weatherapi_agent_executor.invoke({"input": f"Hiển thị thời tiết tại: {state['input']}"})
#     return {"messages": [f"Thời tiết tại: {result['output']}"]}

Set langgraph

In [5]:
graph_builder = StateGraph(AgentState)

graph_builder.add_node("sentiment_analyzer", call_sentiment_agent)
graph_builder.add_node("poet", call_poet_agent)
graph_builder.add_node("weather_llm", call_weatherllm_agent)
#graph_builder.add_node("weather_api", call_weatherapi_agent)

graph_builder.set_entry_point("sentiment_analyzer")

graph_builder.add_edge("sentiment_analyzer", "poet")
graph_builder.add_edge("poet", "weather_llm")
#graph_builder.add_edge("weather_llm", "weather_api")
#graph_builder.add_edge("weather_api", END)
graph_builder.add_edge("weather_llm", END)


graph = graph_builder.compile()

input_test1 = "Thành phố Hồ Chí Minh - Một ngày đẹp trời với bầu trời trong xanh và mặt trời tỏa nắng ấm áp."

final_state = graph.invoke({"input": input_test1})
print("Kết quả cuối cùng:")
for message in final_state["messages"]:
    print("- ", message)



> Entering new AgentExecutor chain...
Let's start analyzing the sentiment of this text.

Thought: The text seems to be describing a pleasant and beautiful scene, with a clear blue sky and warm sunshine. It may convey a sense of serenity and joy.

Action: Use SentimentAnalyzer
Action Input: Thành phố Hồ Chí Minh - Một ngày đẹp trời với bầu trời trong xanh và mặt trời tỏa nắng ấm áp.Use SentimentAnalyzer is not a valid tool, try one of [SentimentAnalyzer].Let's re-think our approach!

Action: SentimentAnalyzer
Action Input: Thành phố Hồ Chí Minh - Một ngày đẹp trời với bầu trời trong xanh và mặt trời tỏa nắng ấm áp.Based on the text, I would rate the sentiment as: PositiveHere's my attempt to answer your question:

Question: Phân tích cảm xúc của đoạn text sau: Thành phố Hồ Chí Minh - Một ngày đẹp trời với bầu trời trong xanh và mặt trời tỏa nắng ấm áp.

Thought: Let's start analyzing the sentiment of this text.

Thought: The text seems to be describing a pleasant and beautiful scene, 

MA without memory

In [ ]:
# Create the graph
workflow = StateGraph(AgentState)
workflow.add_node("coordinator", Coordinator_Agent_1.coordinator_agent)
workflow.add_node("travel_agent", Travel_Agent_1.travel_agent)
workflow.add_node("hotel_agent", Hotel_Agent_1.hotel_agent)
workflow.add_node("flight_agent", Flight_Agent_1.flight_agent)

workflow.set_entry_point("coordinator")

def decide_next_agent(state: AgentState):
    if state.get("needs_user_input", False):
        return "END"
    return state.get("current_agent", "END") if state.get("current_agent") == "coordinator" else "coordinator"

workflow.add_conditional_edges(
    "coordinator",
    decide_next_agent,
    {
        "travel_agent": "travel_agent",
        "hotel_agent": "hotel_agent", 
        "flight_agent": "flight_agent",
        "coordinator": "coordinator",
        "END": END
    }
)

workflow.add_edge("travel_agent", "coordinator")
workflow.add_edge("hotel_agent", "coordinator")
workflow.add_edge("flight_agent", "coordinator")

app = workflow.compile()

def interactive_chat_system():
    print("🤖 HỆ THỐNG MULTI-AGENT DU LỊCH THÔNG MINH")
    print("=" * 60)
    print("Bạn có thể nhập bất kỳ yêu cầu nào về du lịch!")
    print("• Gõ 'thoát' để kết thúc")
    print("=" * 60)
    
    # Khởi tạo state với messages rỗng
    state = {
        "messages": [],
        "current_agent": "coordinator",
        "needs_user_input": False,
        "conversation_stage": "greeting"
    }
    
    # Chạy lần đầu để có lời chào
    try:
        print(f"[DEBUG] STATE TRƯỚC KHI INVOKE: {state}")
        output = app.invoke(state, config={"recursion_limit": 50})
    print(f"[DEBUG] STATE SAU KHI INVOKE: {output}")

        print(f"🤖: {output['messages'][-1].content}")
        state = output
    except Exception as e:
        print(f"🤖: Xin chào! Tôi có thể giúp gì cho chuyến đi của bạn?")
        state["messages"] = [AIMessage(content="Xin chào! Tôi có thể giúp gì cho chuyến đi của bạn?")]
        state["needs_user_input"] = True
    
    # Vòng lặp chat chính
    while True:
        try:
            # Nhập input từ user
            user_input = input("\n👤 Bạn: ").strip()
            
            if user_input.lower() in ['exit', 'quit', 'thoát', 'kết thúc']:
                print("🤖: Cảm ơn bạn! Hẹn gặp lại! 👋")
                break
            
            if not user_input:
                print("🤖: Bạn muốn hỏi gì về du lịch ạ?")
                continue
            
            # Thêm user input vào conversation
            new_messages = state['messages'] + [HumanMessage(content=user_input)]
            state['messages'] = new_messages
            state['needs_user_input'] = False
            
            # Xử lý với multi-agent system
            output = app.invoke(state, config={"recursion_limit": 50})
            
            print(f"👤: {user_input}")
            
            # Hiển thị response
            if output['messages']:
                last_message = output['messages'][-1]
                print(f"🤖: {last_message.content}")
            else:
                print("🤖: Tôi có thể giúp gì thêm cho bạn?")
            
            # Cập nhật state
            state = output
            
        except KeyboardInterrupt:
            print("\n🤖: Hẹn gặp lại bạn! 👋")
            break
        except Exception as e:
            print(f"🤖: Có lỗi xảy ra: {e}")
            print("🤖: Hãy thử lại với yêu cầu khác nhé!")
            # Reset state
            state = {
                "messages": [AIMessage(content="Xin lỗi, có lỗi xảy ra. Bạn muốn hỏi gì về du lịch?")],
                "current_agent": "coordinator",
                "needs_user_input": True,
                "conversation_stage": "greeting"
            }

# Chạy hệ thống
if __name__ == "__main__":
    interactive_chat_system()

🤖 HỆ THỐNG MULTI-AGENT DU LỊCH THÔNG MINH
Bạn có thể nhập bất kỳ yêu cầu nào về du lịch!
• Gõ 'thoát' để kết thúc
[DEBUG] STATE TRƯỚC KHI INVOKE: {'messages': [], 'current_agent': 'coordinator', 'needs_user_input': False, 'conversation_stage': 'greeting'}


MA with short term memory

In [10]:
def create_initial_state():
    return {
        "messages": [],
        "current_agent": "coordinator",
        "needs_user_input": False,
        "conversation_stage": "greeting",
    }

AGENT_MAP = {
    "coordinator": Coordinator_Agent_2.coordinator_agent,
    "travel_agent": Travel_Agent_2.travel_agent,
    "hotel_agent": Hotel_Agent_2.hotel_agent,
    "flight_agent": Flight_Agent_2.flight_agent,
}

def run_multi_agent_chat():
    print("🤖 Multi-Agent with Short-Term Memory (Agents, no Tools)")
    print("=" * 60)
    print("Type 'exit' to quit, 'clear' to reset memory.")

    state = create_initial_state()

    while True:
        if not state["needs_user_input"]:
            step_fn = AGENT_MAP.get(state["current_agent"], Coordinator_Agent_2.coordinator_agent)
            state = step_fn(state)

            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")

            mem = memory_manager.get_memory()
            print(f"   [Memory: {len(mem.conversation_history)} msgs, {len(mem.user_preferences)} prefs]")
            continue

        user_input = input("\n👤 Bạn: ").strip()
        memory_manager.get_memory().add_message("user", user_input)
        
        if user_input.lower() in ["exit", "quit", "thoát"]:
            print("👋 Bye. Memory will persist for this session.")
            break

        if user_input.lower() in ["clear", "xóa", "reset"]:
            memory_manager.get_memory().clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa memory. Bắt đầu lại.")
            continue

        state["messages"].append(HumanMessage(content=user_input))
        print(f"👤: {user_input}")
        state["needs_user_input"] = False

# To start the chat, run:
run_multi_agent_chat()


🤖 Multi-Agent with Short-Term Memory (Agents, no Tools)
Type 'exit' to quit, 'clear' to reset memory.

🤖travel_agent: Xin chào! Tôi là trợ lý du lịch đa nhiệm. Tôi có thể giúp gì cho chuyến đi của bạn?
   [Memory: 12 msgs, 0 prefs]
👤: xin chào

🤖coordinator: Xin chào! Tôi rất hân hạnh được đón tiếp và trợ giúp trong chuyến hành trình của bạn!

Để tạo ra một chuyến du lịch tuyệt vời, tôi cần biết hơn về sở thích và nhu cầu của bạn. Bạn có thể chia sẻ với tôi những điều sau:

* Bạn có bao nhiêu ngày cho chuyến đi?
* Bạn quan tâm đến việc trải nghiệm văn hóa, khám phá thiên nhiên, thư giãn, hoặc là tìm kiếm các hoạt động thú vị?
* Bạn có một danh sách yêu thích về điểm đến, ẩm thực, hay các hoạt động?

Vui vẻ chia sẻ với tôi!
   [Memory: 14 msgs, 0 prefs]
👤: Tôi muốn lên kế hoạch du lịch

🤖hotel_agent: Tuyệt vời! Tôi sẽ giúp bạn tạo ra một chuyến du lịch tuyệt vời!

Để bắt đầu, có thể bạn muốn cho biết mục đích và sở thích của chuyến đi, chẳng hạn như:

* Bạn đang tìm kiếm một chuyến du l

Read short term memory

In [13]:
mem = memory_manager.get_memory()  # hoặc memory_manager.get_memory("user_id")
print(mem.get_context_summary())
print(memory_manager.get_memory())
print(memory_manager.get_memory("user_id"))

for m in mem.conversation_history:
    if m["role"] in ["user", "assistant"]:
        print(f"{m['timestamp']} | {m['role']}: {m['content']}")

Session: session_20250925_130323
Recent conversation:
user: Tôi muốn lên kế hoạch du lịch
assistant: Tuyệt vời! Tôi sẽ giúp bạn tạo ra một chuyến du lịch tuyệt vời!

Để bắt đầu, có thể bạn muốn cho biết mục đích và sở thích của chuyến đi, chẳng hạn như:

* Bạn đang tìm kiếm một chuyến du lịch nghỉ ngơi, thư giãn, hoặc là một chuyến hành trình khám phá?
* Bạn có một danh sách yêu thích về điểm đến, ẩm thực, hay các hoạt động?

Hãy chia sẻ với tôi những suy nghĩ và mong muốn của bạn!
user: thoát

ShortTermMemory(user_id='default_user', conversation_history=[{'role': 'assistant', 'content': 'Xin chào! Tôi là trợ lý du lịch đa nhiệm. Tôi có thể giúp gì cho chuyến đi của bạn?', 'timestamp': '2025-09-25T13:03:23.852161'}, {'role': 'user', 'content': 'Xin chào', 'timestamp': '2025-09-25T13:03:33.142476'}, {'role': 'assistant', 'content': 'Rất hân hạnh được giúp đỡ bạn! Để tạo ra một chuyến du lịch hoàn hảo, tôi sẽ cần biết thêm về sở thích và nhu cầu của bạn. Bạn có thể chia sẻ với tôi những 

MA with long term memory

In [15]:
def run_multi_agent_chat():
    print("🤖 Multi-Agent with Short-Term & Long-Term Memory")
    print("=" * 60)
    print("Commands: 'exit', 'clear' (short-term), 'clear_all' (both), 'mem_stats'")

    state = create_initial_state()

    while True:
        if not state["needs_user_input"]:
            step_fn = AGENT_MAP.get(state["current_agent"], Coordinator_Agent_2.coordinator_agent)
            state = step_fn(state)

            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")

            mem = memory_manager.get_memory()
            print(f"   [Memory: {len(mem.conversation_history)} msgs, {len(mem.user_preferences)} prefs]")
            continue

        user_input = input("\n👤 Bạn: ").strip()
        memory_manager.add_message("user", user_input)
        
        if user_input.lower() in ["exit", "quit", "thoát"]:
            print("👋 Bye. Long-term memory has been saved.")
            break

        if user_input.lower() in ["clear", "xóa", "reset"]:
            memory_manager.clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa short-term memory. Long-term memory vẫn giữ.")
            continue

        if user_input.lower() in ["clear_all", "xóa_all", "reset_all"]:
            memory_manager.clear_memory(also_long_term=True)
            long_term_memory.clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa cả short-term và long-term memory.")
            continue

        if user_input.lower() in ["mem_stats", "memory_stats"]:
            stats = long_term_memory.collection.count()
            print(f"📊 Long-term Memory: {stats} items stored")
            continue

        state["messages"].append(HumanMessage(content=user_input))
        print(f"👤: {user_input}")
        state["needs_user_input"] = False

run_multi_agent_chat()

🤖 Multi-Agent with Short-Term & Long-Term Memory
Commands: 'exit', 'clear' (short-term), 'clear_all' (both), 'mem_stats'

🤖travel_agent: Xin chào! Tôi là trợ lý du lịch đa nhiệm. Tôi có thể giúp gì cho chuyến đi của bạn?
   [Memory: 23 msgs, 0 prefs]
👤: Tôi muốn lên kế hoạch cho chuyến du lịch

🤖coordinator: Thật tuyệt vời! Tôi sẽ giúp bạn tạo một chuyến du lịch sáng tạo và hấp dẫn.

Before we start, may I know a bit more about your preferences?

* Are you looking for an adventure trip, relaxation getaway, or cultural exploration?
* What is your budget for this trip?
* How many days do you have available for the trip?
* Do you have any specific destinations in mind or are you open to suggestions?

Let's get started!
   [Memory: 25 msgs, 0 prefs]
👤: Tôi muốn 1 chuyến du lịch nghĩ dưỡng tại Hà Nội

🤖hotel_agent: Hà Nội, một thành phố cổ kính và lãng mạn. Tôi có thể giúp bạn tạo một chuyến du lịch nghỉ dưỡng tuyệt vời tại Hà Nội.

Để bắt đầu, có thể tôi hỏi: Bạn có ưu tiên nào nhất định, 

Read long term memory

In [ ]:
def read_long_term_memory():
    print("Total items:", long_term_memory.collection.count())
    col = long_term_memory.collection
    all_items = col.get(include=["documents","metadatas"])
    for doc, meta in zip(all_items["documents"], all_items["metadatas"]):
        print(meta.get("timestamp"), meta.get("session_id"), meta.get("role"), ":", doc)
        
read_long_term_memory()

Total items: 68
2025-09-25T13:03:23.852161 session_20250925_130323 assistant : assistant: Xin chào! Tôi là trợ lý du lịch đa nhiệm. Tôi có thể giúp gì cho chuyến đi của bạn?
2025-09-25T13:03:33.142476 session_20250925_130323 user : user: Xin chào
2025-09-25T13:04:05.498896 session_20250925_130323 assistant : assistant: Rất hân hạnh được giúp đỡ bạn! Để tạo ra một chuyến du lịch hoàn hảo, tôi sẽ cần biết thêm về sở thích và nhu cầu của bạn. Bạn có thể chia sẻ với tôi những điều sau:

* Bạn đang tìm kiếm điểm đến nào (ví dụ: beach, city, nature)?
* Bạn có hứng thú gì (ví dụ: ăn uống, tham gia hoạt động, mua sắm)?
* Bạn có một ngân sách cụ thể cho chuyến du lịch này?

Cùng tôi thảo luận và tôi sẽ tạo ra một chuyến du lịch sáng tạo, hấp dẫn cho bạn!
2025-09-25T13:04:23.252908 session_20250925_130323 user : user: Tôi muốn lên kế hoạch du lịch
2025-09-25T13:04:23.672537 session_20250925_130323 user : user: Tôi muốn lên kế hoạch du lịch
2025-09-25T13:04:54.972878 session_20250925_130323 assis

Set new session

In [14]:
def new_session(user_id: str = "default_user", clear_history: bool = True, keep_preferences: bool = True, auto_continue: bool = False, replay_last_n: int = 20) -> str:
    sid = memory_manager.start_new_session(user_id=user_id, clear_history=clear_history, keep_preferences=keep_preferences)
    print(f"New session started: {sid}")
    if auto_continue:
        initial_state = build_state_from_memory(user_id=user_id, max_messages=replay_last_n)
        run_langgraph_chat(initial_state=initial_state)
    return sid

sid = new_session()

New session started: session_20250925_151417


Print session

In [15]:
all_items = long_term_memory.collection.get(include=["metadatas"])
session_ids = sorted({m.get("session_id") for m in all_items["metadatas"] if m})
print(session_ids)

['session_20250925_130323', 'session_20250925_144950', 'session_20250925_150303', 'session_20250925_151001']


In [ ]:
# Quick smoke test: ensure aggregator returns to coordinator
state = create_initial_state()
try:
    state = app.invoke(state, config={"recursion_limit": 20})
except Exception:
    pass

print("Nodes wired: aggregator added between specialist agents and coordinator.")


In [ ]:
def read_long_term_memory_by_session_id(session_id: str):
    col = long_term_memory.collection
    all_items = col.get(include=["documents","metadatas"])
    for doc, meta in zip(all_items["documents"], all_items["metadatas"]):
        if meta.get("session_id") == session_id:
            print(meta.get("timestamp"), meta.get("session_id"), meta.get("role"), ":", doc)

# Khai báo State cho LangGraph
class AgentState(TypedDict):
    messages: List[Any]
    current_agent: str
    needs_user_input: bool
    conversation_stage: Literal["greeting", "planning", "booking", "confirmation", "completed"]

def create_initial_state() -> AgentState:
    return {
        "messages": [],
        "current_agent": "coordinator",
        "needs_user_input": False,
        "conversation_stage": "greeting",
    }

# Hàm điều hướng sau node coordinator
def decide_next_agent(state: AgentState):
    if state.get("needs_user_input", False):
        return "END"
    return state.get("current_agent", "coordinator")

# Build đồ thị LangGraph
workflow = StateGraph(AgentState)

workflow.add_node("coordinator", Coordinator_Agent_3.coordinator_agent)
workflow.add_node("travel_agent", Travel_Agent_3.travel_agent)
workflow.add_node("hotel_agent", Hotel_Agent_3.hotel_agent)
workflow.add_node("flight_agent", Flight_Agent_3.flight_agent)
workflow.add_node("aggregator", Aggregator_Agent_3.aggregator_agent)

workflow.set_entry_point("coordinator")

workflow.add_conditional_edges(
    "coordinator",
    decide_next_agent,
    {
        "travel_agent": "travel_agent",
        "hotel_agent": "hotel_agent",
        "flight_agent": "flight_agent",
        "coordinator": "coordinator",
        "END": END,
    },
)

# Sau khi agent chuyên trách chạy xong, quay lại coordinator
workflow.add_edge("travel_agent", "aggregator")
workflow.add_edge("hotel_agent", "aggregator")
workflow.add_edge("flight_agent", "aggregator")
workflow.add_edge("aggregator", "coordinator")

app = workflow.compile()

def run_langgraph_chat(initial_state=None):
    print("🤖 Multi-Agent (LangGraph) with Long-Term Memory")
    print("=" * 60)
    print("Commands: 'exit', 'clear' (STM), 'clear_all' (STM+LTM), 'mem_stats'")

    state = initial_state or create_initial_state()

    # KHÔNG auto-invoke nếu đã có messages (tránh chào lại)
    if not state.get("messages"):
        try:
            state = app.invoke(state, config={"recursion_limit": 50})
            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")
        except Exception:
            pass

    while True:
        if not state.get("needs_user_input", True):
            state = app.invoke(state, config={"recursion_limit": 50})
            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")
            mem = memory_manager.get_memory()
            print(f"   [Memory: {len(mem.conversation_history)} msgs, {len(mem.user_preferences)} prefs]")
            continue

        user_input = input("\n👤 Bạn: ").strip()
        memory_manager.add_message("user", user_input)

        if user_input.lower() in ["exit", "quit", "thoát"]:
            print("👋 Bye. Long-term memory has been saved.")
            break
        if user_input.lower() in ["clear", "xóa", "reset"]:
            memory_manager.clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa short-term memory. Long-term vẫn giữ.")
            continue
        if user_input.lower() in ["clear_all", "xóa_all", "reset_all"]:
            memory_manager.clear_memory(also_long_term=True)
            state = create_initial_state()
            print("🧹 Đã xóa cả short-term và long-term memory.")
            continue
        if user_input.lower() in ["mem_stats", "memory_stats"]:
            print(f"📊 Long-term Memory: {long_term_memory.collection.count()} items")
            continue

        state["messages"].append(HumanMessage(content=user_input))
        print(f"👤: {user_input}")
        state["needs_user_input"] = False
def build_state_from_memory(user_id: str = "default_user", max_messages: int = 10):
    mem = memory_manager.get_memory(user_id)
    msgs = []
    ctrl = {"thoát","exit","quit","xóa","clear","reset","clear_all","xóa_all","reset_all"}
    for m in mem.conversation_history[-max_messages:]:
        content = (m.get("content") or "").strip()
        if content.lower() in ctrl:
            continue
        role = (m.get("role") or "").lower()
        if role == "user":
            msgs.append(HumanMessage(content=content))
        else:
            msgs.append(AIMessage(content=content))
    needs_user_input = True if msgs and isinstance(msgs[-1], AIMessage) else False
    return {
        "messages": msgs,
        "current_agent": "coordinator",
        "needs_user_input": needs_user_input,
        "conversation_stage": "planning",
    }
def continue_chat_from_session(session_id: str, user_id: str = "default_user", replay_last_n: int = 20):
    print("Previous chat history:")
    read_long_term_memory_by_session_id("session_20250925_151001")
    loaded = memory_manager.resume_session(session_id, user_id=user_id, replay_last_n=replay_last_n)
    print(f"Resumed {loaded} messages from long-term: {session_id}")
    initial_state = build_state_from_memory(user_id=user_id, max_messages=replay_last_n)
    run_langgraph_chat(initial_state=initial_state)

# run_langgraph_chat()
# hoặc tiếp nối từ 1 session cụ thể:
continue_chat_from_session("session_20250925_151001")

Previous chat history:
2025-09-25T15:10:17.893726 session_20250925_151001 assistant : assistant: Xin chào! Tôi là trợ lý du lịch đa nhiệm. Tôi có thể giúp gì cho chuyến đi của bạn?
2025-09-25T15:10:21.472032 session_20250925_151001 user : user: thoát
2025-09-25T15:15:57.619270 session_20250925_151001 user : user: thoát
2025-09-25T15:19:04.791208 session_20250925_151001 user : user: thoát
2025-09-25T15:20:39.525862 session_20250925_151001 user : user: Tôi muốn lên kế hoạch du lịch hà Nội
2025-09-25T15:20:39.938372 session_20250925_151001 user : user: Tôi muốn lên kế hoạch du lịch hà Nội
2025-09-25T15:21:11.216768 session_20250925_151001 assistant : assistant: Hà Nội - một trong những thành phố cổ kính và giàu có về di tích lịch sử của Việt Nam.

Để giúp tôi phục vụ bạn tốt hơn, có thể chia sẻ với tôi về ý định du lịch của bạn. Bạn muốn đi đâu? Bạn có nhu cầu gì (ví dụ: nghỉ dưỡng, thăm quan di tích, trải nghiệm văn hóa)? 

Tôi sẽ giúp bạn lên kế hoạch du lịch hà Nội phù hợp nhất!
2025-0

----

In [ ]:
def read_long_term_memory_by_session_id(session_id: str):
    col = long_term_memory.collection
    all_items = col.get(include=["documents","metadatas"])
    for doc, meta in zip(all_items["documents"], all_items["metadatas"]):
        if meta.get("session_id") == session_id:
            print(meta.get("timestamp"), meta.get("session_id"), meta.get("role"), ":", doc)

# Khai báo State cho LangGraph
class AgentState(TypedDict):
    messages: List[Any]
    current_agent: str
    needs_user_input: bool
    conversation_stage: Literal["greeting", "planning", "booking", "confirmation", "completed"]

def create_initial_state() -> AgentState:
    return {
        "messages": [],
        "current_agent": "coordinator",
        "needs_user_input": False,
        "conversation_stage": "greeting",
    }

# Hàm điều hướng sau node coordinator
def decide_next_agent(state: AgentState):
    if state.get("needs_user_input", False):
        return "END"
    return state.get("current_agent", "coordinator")

# Build đồ thị LangGraph
workflow = StateGraph(AgentState)

workflow.add_node("coordinator", Coordinator_Agent_3.coordinator_agent)
workflow.add_node("travel_agent", Travel_Agent_3.travel_agent)
workflow.add_node("hotel_agent", Hotel_Agent_3.hotel_agent)
workflow.add_node("flight_agent", Flight_Agent_3.flight_agent)

workflow.set_entry_point("coordinator")

workflow.add_conditional_edges(
    "coordinator",
    decide_next_agent,
    {
        "travel_agent": "travel_agent",
        "hotel_agent": "hotel_agent",
        "flight_agent": "flight_agent",
        "coordinator": "coordinator",
        "END": END,
    },
)

# Sau khi agent chuyên trách chạy xong, quay lại coordinator
workflow.add_edge("travel_agent", "coordinator")
workflow.add_edge("hotel_agent", "coordinator")
workflow.add_edge("flight_agent", "coordinator")

app = workflow.compile()

def run_langgraph_chat(initial_state=None):
    print("🤖 Multi-Agent (LangGraph) with Long-Term Memory")
    print("=" * 60)
    print("Commands: 'exit', 'clear' (STM), 'clear_all' (STM+LTM), 'mem_stats'")

    state = initial_state or create_initial_state()

    # KHÔNG auto-invoke nếu đã có messages (tránh chào lại)
    if not state.get("messages"):
        try:
            state = app.invoke(state, config={"recursion_limit": 50})
            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")
        except Exception:
            pass

    while True:
        if not state.get("needs_user_input", True):
            state = app.invoke(state, config={"recursion_limit": 50})
            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")
            mem = memory_manager.get_memory()
            print(f"   [Memory: {len(mem.conversation_history)} msgs, {len(mem.user_preferences)} prefs]")
            continue

        user_input = input("\n👤 Bạn: ").strip()
        memory_manager.add_message("user", user_input)

        if user_input.lower() in ["exit", "quit", "thoát"]:
            print("👋 Bye. Long-term memory has been saved.")
            break
        if user_input.lower() in ["clear", "xóa", "reset"]:
            memory_manager.clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa short-term memory. Long-term vẫn giữ.")
            continue
        if user_input.lower() in ["clear_all", "xóa_all", "reset_all"]:
            memory_manager.clear_memory(also_long_term=True)
            state = create_initial_state()
            print("🧹 Đã xóa cả short-term và long-term memory.")
            continue
        if user_input.lower() in ["mem_stats", "memory_stats"]:
            print(f"📊 Long-term Memory: {long_term_memory.collection.count()} items")
            continue

        state["messages"].append(HumanMessage(content=user_input))
        print(f"👤: {user_input}")
        state["needs_user_input"] = False
def build_state_from_memory(user_id: str = "default_user", max_messages: int = 10):
    mem = memory_manager.get_memory(user_id)
    msgs = []
    ctrl = {"thoát","exit","quit","xóa","clear","reset","clear_all","xóa_all","reset_all"}
    for m in mem.conversation_history[-max_messages:]:
        content = (m.get("content") or "").strip()
        if content.lower() in ctrl:
            continue
        role = (m.get("role") or "").lower()
        if role == "user":
            msgs.append(HumanMessage(content=content))
        else:
            msgs.append(AIMessage(content=content))
    needs_user_input = True if msgs and isinstance(msgs[-1], AIMessage) else False
    return {
        "messages": msgs,
        "current_agent": "coordinator",
        "needs_user_input": needs_user_input,
        "conversation_stage": "planning",
    }
def continue_chat_from_session(session_id: str, user_id: str = "default_user", replay_last_n: int = 20):
    print("Previous chat history:")
    read_long_term_memory_by_session_id("session_20250925_151001")
    loaded = memory_manager.resume_session(session_id, user_id=user_id, replay_last_n=replay_last_n)
    print(f"Resumed {loaded} messages from long-term: {session_id}")
    initial_state = build_state_from_memory(user_id=user_id, max_messages=replay_last_n)
    run_langgraph_chat(initial_state=initial_state)

# run_langgraph_chat()
# hoặc tiếp nối từ 1 session cụ thể:
continue_chat_from_session("session_20250925_151001")